<a href="https://colab.research.google.com/github/jinseriouspark/embedding_for_all/blob/main/%5Bw1%5D_tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenizers
1. SentencePiece
2. BPE from scratch (Sentencepiece paper : [link](https://https://aclanthology.org/P16-1162/))
3. Morphem BPE

## 1. SentencePiece


- 참고자료 : https://devocean.sk.com/blog/techBoardDetail.do?ID=164570&boardType=techBlog
-https://process-mining.tistory.com/191

In [1]:
!pip install sentencepiece
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00


### Training & inference & save & upload

- corpus를 subword단위로 쪼갠 다음, 빈도수를 계산해 높은 빈도로 함께 등장한 subword를 병합해 학습할 거에요.
- 적절한 subword 를 파악하기 위해서는 사전에 정리된 corpus도 필요해요.
- 이미 만들어져있는 nsmc를 사용하려고 합니다. (허깅페이스 데이터셋에서도 다운받을 수 있어요)
  - 링크1 : https://github.com/e9t/nsmc
  - 링크2 : https://huggingface.co/datasets/nsmc

In [2]:
from tqdm import tqdm
from datasets import load_dataset
import os

data_dir = './data'
dataset = load_dataset('nsmc')
os.makedirs(data_dir, exist_ok = True)
for split_key in dataset.keys():
  doc_path = f'{data_dir}/{split_key}.txt'
  with open(doc_path, 'w') as f:
    for doc in dataset[split_key]['document']:
      f.write(doc + '\n')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
def load_data(file_path):
  with open(file_path, 'r', encoding = 'utf-8') as f:
    data = f.readlines()
  return [d.strip() for d in data]

korean_train = load_data('/content/data/train.txt')
korean_train[:10]

korean_test = load_data('/content/data/test.txt')
korean_test[:10]

['굳 ㅋ',
 'GDNTOPCLASSINTHECLUB',
 '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아',
 '지루하지는 않은데 완전 막장임... 돈주고 보기에는....',
 '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??',
 '음악이 주가 된, 최고의 음악영화',
 '진정한 쓰레기',
 '마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다',
 '갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다',
 '이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..']

In [4]:
# BBPE를 Sentencepiece 라이브러리를 활용해 학습해보기
import sentencepiece as spm
from pathlib import Path

data_dir = './data'
path = [str(x) for x in Path(data_dir).glob('*.txt')]

In [5]:
corpus = ','.join(path)

In [6]:
prefix = 'sp-nsmc-test'
vocab_size = 31900 - 7
# vocab_size 는 10_000 ~ 52_000 사이이며 모델에 따라 다르다고 합니다.
# subwords token 31_900개
# special token 7개 제외
# additional_special_tokens (T5용 <extra_id_XX> 토큰) 100개
# -> 총 32_000개

In [7]:
spm.SentencePieceTrainer.train(
     f' --input={corpus} --model_prefix={prefix}' +
     f' --vocab_size={vocab_size + 7}' + #  최종 데이터셋 갯수
      ' --model_type=bpe' + # 어떤 토크나이저를 사요할 것인지 설정 : unigram, bpe, char, word
      ' --max_sentence_length=999_999' +
      ' --pad_id=0 --pad_piece=<pad>' +
      ' --unk_id=1 --unk_piece=<unk>' +
      ' --bos_id=2 --bos_piece=<s>' +
      ' --eos_id=3 --eos_piece=</s>' +
      ' --user_defined_symbols=<sep>,<cls>,<mask>' +
      ' --byte_fallback=True'
)


In [8]:
# 학습 완료 시 .model 과 .vocab 파일을 얻을 수 있음

In [9]:
# 사용방법1 :
model_name = 'sp-nsmc-test'
sp = spm.SentencePieceProcessor(model_file=f'{model_name}.model')
text ='같은 배우여도 독립영화냐 상업영화냐에 따라 생김새가 조금씩 다르게 보여'
print(sp.encode_as_pieces(text))
print(sp.encode_as_ids(text))

['▁같은', '▁배우', '여도', '▁독립영화', '냐', '▁상업', '영화냐', '에', '▁따라', '▁생김', '새가', '▁조금씩', '▁다르게', '▁보여']
[609, 388, 7909, 3784, 30389, 3606, 9689, 30189, 1412, 14885, 19366, 9729, 5396, 589]


In [10]:
# 사용방법2 : T5Tokenizer 로 매핑한 허깅페이스에 업로드 및 다운로드로 활용

# 1. t5로 매핑
from transformers import T5Tokenizer
model_name = 'sp-nsmc-test'
tokenizer = T5Tokenizer(vocab_file = f'{model_name}.model')
tokenizer.save_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


('sp-nsmc-test/tokenizer_config.json',
 'sp-nsmc-test/special_tokens_map.json',
 'sp-nsmc-test/spiece.model',
 'sp-nsmc-test/added_tokens.json')

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
# 2. 허깅페이스에 업로드
hf_spacename = 'seriouspark' # 허깅페이스 아이디 입력해주세요
hf_model_name = f'{hf_spacename}/{model_name}'
tokenizer.push_to_hub(hf_model_name)

# 3. 토크나이저 다운로드
from transformers import AutoTokenizer
hf_tokenizer = AutoTokenizer.from_pretrained(hf_model_name)

ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

In [ ]:
# 위와 같이 매핑한 다음 쉽게 결과물을 확인할 수 있어요
text ='같은 배우여도 독립영화냐 상업영화냐에 따라 생김새가 조금씩 다르게 보여'
display(tokenizer.tokenize(text))
display(tokenizer(text)['input_ids'])

In [ ]:
# 위와 같이 매핑한 다음 쉽게 결과물을 확인할 수 있어요
text ='같은 배우여도 독립영화냐 상업영화냐에 따라 생김새가 조금씩 다르게 보여'
display(hf_tokenizer.tokenize(text))
display(hf_tokenizer(text)['input_ids'])

##2. BPE from Scratch
- 하지만 구현은 byte 가 아니라 char 기준입니다

1. 빈도 계산
2. 모든 단어를 글자 단위로 분리
3. 병합

In [ ]:
import re
from collections import defaultdict



### 1. 빈도 계산
: 문장 내 아래 단어 발생 빈도를 집계

  문장 : 나는 **계란밥**을 먹는다. **계란밥**! 얼마나 맛있던가. 나는 **계란밥**이 세상에서 제일좋다. 물론 **간장밥** 러버도 있겠지. 예를 들어 내 동생은 **간장밥** 하나만 있으면 1주일을 버틸 수 있다. 어떻게 하면 **간장밥** 그 단일 메뉴가 아침을 해결할 수 있을까. 말이 안된다. 중학생 때 먹은 **간장밥**, 그게 내 마지막 **간장밥**이다.

  나는 내동생과 서로를 이해하지 못했다. 어느날 부모님이 계란과 간장을 섞어 **간장계란밥**을 만들었다. 노란색의 밥에 검정 소스가 있는, 이상한 그 메뉴의 이름을 묻자 부모님은 웃으며 내게 말했다. '**간장계란밥**'

  **볶음밥**이야? 라고 물어봤지만 부모님은 고개를 저었다. 아니 **볶음밥**이라니? 이건 기름에 볶지 않았어. **볶음밥**의 기본은 볶음인걸? 이건 간장과 계란이 합쳐진 비빔밥이야.


    < dictionary>
    : 훈련데이터에 있는 단어와 등장 빈도수
    - '계란밥': 3
    - '간장밥' : 5
    - '간장계란밥' : 2
    - '볶음밥' : 3

      <vocabulary>
    - 계란밥
    - 간장밥
    - 간장계란밥
    - 볶음밥


###2. 분리:  모든 단어를 글자 단위로 나눔(char)


    < dictionary>
    -  계 란 밥 : 3
        - 간 장 밥 : 5
        - 간 장 계 란 밥 : 2
        - 볶 음 밥 : 3
      
      < vocab>
    - 계, 란, 밥, 간, 장, 볶, 음


### 3. 병합
: 몇 번 merge 해서 하나의 유니그램으로 통합할 것인가?
num_merges = 10 -> 총 10회 반복한다


  1회 : 빈도수가 5 + 2= 7 인 (간, 장) 의 쌍을 간장 으로 통합

    - 계 란 밥 : 3
    - 간장 밥: 5
    - 간장 계 란 밥 : 2
    - 볶 음 밥 : 3

  2회 : 빈도수가 2 + 3 = 5로 가장 높은 (계, 란) 을 계란 으로 통합, (간장, 밥) 을 간장밥 으로 통합

    - 계란 밥: 3
    - 간장밥: 5
    - 간장 계란 밥 : 2
    - 볶 음 밥 : 3

  3회 : 빈도수가 3 + 2 = 5로 가장 높은 (계란, 밥) 을 계란밥으로 통합

    - 계란밥 : 3
    - 간장밥 : 5
    - 간장 계란밥 : 2
    - 볶 음 밥 : 3

  ... 반복




: 3번 반복하였을 때 결과
    
    < dictionary>
    - 계란밥 : 3
    - 간장밥 : 5
    - 간장 계란밥 : 2
    - 볶 음 밥 : 3

    < vocab>
    - 계, 란, 밥, 간, 장, 볶, 음, 계란밥, 간장밥, 간장


만약, 계란간장볶음밥이 등장한다면?
  - 계란 간장 볶 음 밥 으로 분리 되며, 이 모든 것은 단어 집합에 있는 단어이기 때문에 OOV가 아니게 됩니다!




In [ ]:
import re, collections

# Character pair encoding
def get_word_freq(corpus):
  word_freq = collections.defaultdict(int)
  for sentence in corpus:
    for word in sentence.split():
      word_space = ' '.join([w for w in word if len(word)>1])
      word_freq[word_space] += 1
  return word_freq

def get_stats(vocab):
  pairs = collections.defaultdict(int)
  for word, freq in vocab.items():
    symbols = word.split()
    for i in range(len(symbols)-1):
      pairs[symbols[i], symbols[i+1]] += freq
  return pairs

def merge_vocab(pair, v_in):
  v_out = {}
  bigram = re.escape(' '.join(pair))
  p = re.compile(f'(?<!\S)' + bigram + r'(?!\S)')
  for word in v_in:
    w_out = p.sub(''.join(pair), word)
    v_out[w_out] = v_in[word]
  return v_out

In [ ]:
# 위의 예시에 따른 결과값을 확인하기 위해 띄어쓰기를 부자연스럽게 적용하였습니다.
corpus = [
    '나는 계란밥 을 먹는다',
    '계란밥 ! 얼마나 맛있던가.',
    '나는 계란밥 이 세상에서 제일좋다.',
    '물론 간장밥 러버도 있겠지.',
    '예를 들어 내 동생은 간장밥 하나만 있으면 1주일을 버틸 수 있다.',
    '어떻게 하면 간장밥 그 단일 메뉴가 아침을 해결할 수 있을까.',
    '말이 안된다. 중학생 때 먹은 간장밥 , 그게 내 마지막 간장밥 이다.',
    '나는 내동생과 서로를 이해하지 못했다.',
    '어느날 부모님이 계란과 간장을 섞어 간장계란밥 을 만들었다.',
    '노란색의 밥에 검정 소스가 있는, 이상한 그 메뉴의 이름을 묻자 부모님은 웃으며 내게 말했다.',
    '간장계란밥',
    '볶음밥 이야? 라고 물어봤지만 부모님은 고개를 저었다.',
    '아니 볶음밥 이라니? 이건 기름에 볶지 않았어.',
    '볶음밥 의 기본은 볶음인걸? 이건 간장과 계란이 합쳐진 비빔밥이야.'
      ]




In [ ]:
bpe_vocab_history = {}
vocab = get_word_freq(corpus)
num_merges = 10
for i in range(num_merges):
  pairs = get_stats(vocab)
  best = max(pairs, key=pairs.get)
  vocab = merge_vocab(best, vocab)
  bpe_vocab_history[best] = i

In [ ]:
# merge history
bpe_vocab_history

In [ ]:
class BPE_tokenizer:
  def __init__(self, num_merges):
    self.num_merges = num_merges
    self.vocab = None
    self.byte_pairs = None
    self.token_vocab = None
    self.bpe_vocab_history = {}

  # BPE알고리즘 학습
  def train(self, corpus):
    self.vocab = get_word_freq(corpus)
    for i in range(self.num_merges):
      self.byte_pairs = get_stats(self.vocab)
      best = max(self.byte_pairs, key=self.byte_pairs.get)
      self.vocab = merge_vocab(best, self.vocab)
      self.bpe_vocab_history[best] = i

    self.token_vocab = set() # 완료 시 만든 vocab 을 추가해줌
    for word in self.vocab.keys():
      self.token_vocab.update(word.split())

  def tokenize(self, text):
    tokens = text.split()
    tokenized_text = []
    for token in tokens:
      if token in self.token_vocab:
        tokenized_text.append(token)
      else:
        tokenized_text.extend(self.byte_tokenize(token))
    return tokenized_text

  # 텍스트 토큰화
  def byte_tokenize(self, token):
    tokens = []
    while len(token) > 0:
      found = False
      for i in range(len(token), 0, -1):
        subword = token[:i]
        #print(subword)
        if subword in self.token_vocab:
          tokens.append(subword)
          token = token[i:]
          found = True
          break
      if not found :
        tokens.append(token[0])
        token = token[1:]
    return tokens

In [ ]:
# num_merges 는 목표 vocab_size 와 token 품질에 따라 adaptive 하게 적용할 수 있습니다.
# SentencePiece 의 경우 vocab_size = 8000
bpe_tokenizer = BPE_tokenizer(num_merges = 50)
bpe_tokenizer.train(corpus)
print('Token Vocabulary:', bpe_tokenizer.token_vocab)

In [ ]:
text ='동생은 볶음밥 2개랑 탕수육을 먹는다'
print('origin text', text)
print('tokenized text', bpe_tokenizer.tokenize(text))

In [ ]:
# 위의 네이버 영화 리뷰 코퍼스를 활용해 토크나이저 생성
# num_merges 50진행 시, 음절 수준의 토큰이 많이 등장하여, 병합횟수를 늘려 한번 더 수행해보았어요. 더 많이 병합을 해야 할 것 같아요
bpe_tokenizer_nsmc = BPE_tokenizer(num_merges = 2000)
bpe_tokenizer_nsmc.train(korean_train)
print('vocab counts:',len(bpe_tokenizer_nsmc.token_vocab))
print('vocab:', bpe_tokenizer_nsmc.token_vocab)

In [ ]:
# sentencepice 결과물:
# ['▁같은', '▁배우', '여도', '▁독립영화', '냐', '▁상업', '영화냐', '에', '▁따라', '▁생김', '새가', '▁조금씩', '▁다르게', '▁보여']

text ='같은 배우여도 독립영화냐 상업영화냐에 따라 생김새가 조금씩 다르게 보여'
print('origin text', text)
print('tokenized text', bpe_tokenizer_nsmc.tokenize(text))

In [ ]:
import re
import collections

def get_byte_freq(corpus):
    byte_freq = collections.defaultdict(int)
    for sentence in corpus:
        for byte in sentence.encode('utf-8'):  # 문자열을 바이트로 인코딩하여 처리
            byte_freq[byte] += 1
    return byte_freq

def get_stats(byte_vocab):
    pairs = collections.defaultdict(int)
    for byte, freq in byte_vocab.items():
        symbols = [int(byte)]
        for i in range(len(symbols)-1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = pair
    p = re.compile(re.escape(' '.join([chr(b) for b in bigram])))
    for byte in v_in:
        w_out = p.sub(''.join([chr(b) for b in bigram]), byte)
        v_out[w_out] = v_in[byte]
    return v_out

def byte_pair_encoding(corpus, num_merges):
    byte_freq = get_byte_freq(corpus)
    for i in range(num_merges):
        byte_stats = get_stats(byte_freq)
        if not byte_stats:
            break
        best_pair = max(byte_stats, key=byte_stats.get)
        byte_freq = merge_vocab(best_pair, byte_freq)
    return byte_freq

# Example usage

num_merges = 10
corpus
byte_freq = byte_pair_encoding(corpus, num_merges)
print(byte_freq)


#3.  형태소 기반 토크나이저 만들기
- 논문 내 <MorWP-MD> 개념을 차용해 간단하게 구현해보고자 합니다.
- 참고자료 : https://github.com/taeheejeon22/MorphSubDecomp-Korean/blob/main/Tutorial.md


## 프로세스

1. 데이터 수집 및 전처리
2. 자모 분리기 생성
3. Wordpiece Model Training - vocab_size 64k 적용
4. BERT 학습


**WordPiece 토큰화 알고리즘**
- 참고자료 https://huggingface.co/learn/nlp-course/en/chapter6/6

- word piece 는 학습된 규칙이 아니라 최종 어휘만 저장한다는 점에서 BPE 토큰화 방식과 크게 다릅니다.
- WordPiece는 토큰화할 단어부터 시작해, 어휘에 있는 가장 긴 하위 단어를 찾은 다음 이를 분할합니다.

(예시)
단어리스트 : ['나','라','면', '라면', '해물']
신규 단어 : ['라면들']

- wordpiece의 경우, ['라면', '##들']
- BPE의 경우, ['라','면','들'] -> ['라면','##들'] 의 방식처럼 병합을 순서대로 적용하고 토큰화 하여 인코딩된 결과물이 달라질 수 있음




In [17]:
# https://github.com/ratsgo/embedding/blob/master/preprocess/dump.py 참조
# 전처리

import re
def preprocess(sent_lst):
    p_email =  re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
    p_url = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.UNICODE)
    p_wiki_char = re.compile("(\\*$|:$|^파일:.+|^;)", re.UNICODE)
    p_wiki_space =  re.compile("(\\s|゙|゚|　)+", re.UNICODE)
    p_tag = re.compile("^<.+>?", re.UNICODE)

    sent_lst = [re.sub(p_email, " ", sentence) for sentence in sent_lst]
    sent_lst = [re.sub(p_url, " ", sentence) for sentence in sent_lst]
    sent_lst = [re.sub(p_wiki_char, " ", sentence) for sentence in sent_lst]
    sent_lst = [re.sub(p_wiki_space, " ", sentence) for sentence in sent_lst]
    sent_lst = [re.sub(p_tag, " ", sentence) for sentence in sent_lst]

    # our
    p_paren_str = re.compile("\(.+?\)") # 괄호 문자열("(xxx)") 삭제용
    sent_lst = [re.sub(p_paren_str, "", sent) for sent in sent_lst] # 사람(인간)은 짐승(동물)이다 > 사람은 짐승이다

    # kortok
    # p_kakao = re.compile(r"[^가-힣\x20-\x7F]*") # 타 언어 문자, 특수 기호 제거
    p_kakao = re.compile(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\x20-\x7F]*")  # 타 언어 문자, 특수 기호 제거    # 자모 낱글자 살리기
    sent_lst = [re.sub(p_kakao, "", sent) for sent in sent_lst]

    # our
    p_multiple_spaces = re.compile("\s+")   # 무의미한 공백
    sent_lst = [re.sub(p_multiple_spaces, " ", sent) for sent in sent_lst]  # 무의미한 공백을 스페이스(" ")로 치환

    # our
    sent_lst = [sent for sent in sent_lst if not re.search(r"^\s+$", sent)]    # 빈 문장 제거
    sent_lst = [sent.strip() for sent in sent_lst if sent != ""]    # 빈 문장 제거

    # our
    sent_lst = [sent for sent in sent_lst if not (sent.endswith(".") and len(sent.split(" ")) <= 3) ]   # 퇴임 이후.    어린 시절.  생애 후반.  등등의 짧은 라인 없애기

    return sent_lst


In [18]:
def preprocessing_all(data, file_path):
  all_texts = ""
  for ix in tqdm(range(len(data))):
    split_text = data[ix]
    preprocessed_text = preprocess(split_text)
    concat_text = '\n'.join(preprocessed_text)

    if concat_text != "":
      all_texts += (concat_text + '\n\n')

  os.makedirs('morph_token/data', exist_ok = True)
  with open(f'morph_token/data/{file_path}.txt', 'w') as f:
    f.write(all_texts)

In [19]:
preprocessing_all(korean_train, 'preprocessed_train')
#preprocessing_all(korean_test, 'preprocessed_test')

100%|██████████| 150000/150000 [02:38<00:00, 947.55it/s] 


python scripts/mecab_tokenization.py --token_type=morpheme --corpus_path=./corpus/preprocessed/namuwiki_20200302_preprocessed.txt --tokenizer_type=mecab_fixed --decomposition_type=decomposed_lexical --nfd --threads 32


In [21]:
# get pos
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 35.4 MB/s eta 0:00:00


In [20]:
# wordpiece tokenizer
# https://huggingface.co/learn/nlp-course/en/chapter6/6 참고

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
from collections import defaultdict
word_freqs = defaultdict(int)
for text in corpus:
  words_with_offests = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
  new_words = [word for word, offset in words with_]

In [58]:
# 추가적으로 자모 분리해주는 함수
# https://frhyme.github.io/python/python_korean_englished/ 참고
# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


def korean_jamo_spliter(text):
  def is_hangeul(char):
    korean_p = re.compile('[ㄱ-ㅎ가-힣]+')
    return len(re.findall(korean_p, char)) > 0

  r_list = []
  for char in text:
    if is_hangeul(char) :
      cho = (ord(char) - ord('가')) // 588
      jung = ((ord(char) - ord('가')) - (588 * cho)) // 28
      jong = (ord(char) - ord('가')) - (588 * cho) - 28 * jung
      if jong > 0:
        r_list.append([CHOSUNG_LIST[cho], JUNGSUNG_LIST[jung], JONGSUNG_LIST[jong]])
      else:
        r_list.append([CHOSUNG_LIST[cho], JUNGSUNG_LIST[jung]])
    else:
      r_list.append([char])
  return r_list

In [62]:
text = '나 라면 해물라면을 먹었을걸'
def morphem_jamo_spliter(text):
  text_list = []
  kkma= Kkma()
  parsed_text = kkma.pos(text)
  for i in parsed_text:
    if i[1] in (['VV','NNG','EP']):

      result_ = korean_jamo_spliter(i[0])
      text_list.append(sum(result_, []))
    else:
      text_list.append([i[0]])

  return text_list

In [70]:
# 위의 예시에 따른 결과값을 확인하기 위해 띄어쓰기를 부자연스럽게 적용하였습니다.
corpus = [
    '나는 계란밥 을 먹는다',
    '계란밥 ! 얼마나 맛있던가.',
    '나는 계란밥 이 세상에서 제일좋다.',
    '물론 간장밥 러버도 있겠지.',
    '예를 들어 내 동생은 간장밥 하나만 있으면 1주일을 버틸 수 있다.',
    '어떻게 하면 간장밥 그 단일 메뉴가 아침을 해결할 수 있을까.',
    '말이 안된다. 중학생 때 먹은 간장밥 , 그게 내 마지막 간장밥 이다.',
    '나는 내동생과 서로를 이해하지 못했다.',
    '어느날 부모님이 계란과 간장을 섞어 간장계란밥 을 만들었다.',
    '노란색의 밥에 검정 소스가 있는, 이상한 그 메뉴의 이름을 묻자 부모님은 웃으며 내게 말했다.',
    '간장계란밥',
    '볶음밥 이야? 라고 물어봤지만 부모님은 고개를 저었다.',
    '아니 볶음밥 이라니? 이건 기름에 볶지 않았어.',
    '볶음밥 의 기본은 볶음인걸? 이건 간장과 계란이 합쳐진 비빔밥이야.'
      ]




In [71]:

word_freqs = defaultdict(int)
from collections import defaultdict
for text in corpus:
  new_words = morphem_jamo_spliter(text)
  for word in new_words:
    for w in word:
      word_freqs[w] +=1


In [73]:
word_freqs

defaultdict(int,
            {'나': 3,
             '는': 5,
             'ㄱ': 27,
             'ㅖ': 8,
             'ㄹ': 24,
             'ㅏ': 57,
             'ㄴ': 30,
             'ㅂ': 45,
             'ㅇ': 38,
             'ㅡ': 9,
             'ㅁ': 24,
             'ㅓ': 11,
             '다': 8,
             '!': 1,
             '얼마나': 1,
             '맛있': 1,
             '더': 1,
             'ㄴ가': 1,
             '.': 13,
             '이': 9,
             'ㅅ': 13,
             'ㅔ': 3,
             '에서': 1,
             '제일': 1,
             '좋': 1,
             '물론': 1,
             'ㅈ': 13,
             '도': 1,
             'ㅣ': 20,
             'ㅆ': 5,
             '겠': 1,
             '지': 4,
             '를': 3,
             'ㄷ': 8,
             '어': 4,
             '내': 4,
             'ㅗ': 16,
             'ㅐ': 8,
             '은': 5,
             '하나': 1,
             '만': 1,
             '으면': 1,
             '1': 1,
             '주일': 1,
             '을': 4,
             'ㅌ

In [76]:
vocab = list(set(["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + CHOSUNG_LIST.copy() + JUNGSUNG_LIST.copy() + JONGSUNG_LIST.copy()))

In [77]:
splits = {
    word: [c if i == 0 else f]
}

['ㅃ',
 'ㅂ',
 'ㄹ',
 'ㅆ',
 'ㅐ',
 'ㅒ',
 'ㅄ',
 'ㅗ',
 'ㅇ',
 'ㄼ',
 'ㅚ',
 'ㅢ',
 'ㅀ',
 'ㅜ',
 'ㄴ',
 'ㅓ',
 'ㅝ',
 ' ',
 'ㅛ',
 'ㅏ',
 'ㅉ',
 'ㄻ',
 'ㅁ',
 'ㅋ',
 'ㄾ',
 'ㄷ',
 'ㅕ',
 'ㅠ',
 'ㅡ',
 'ㄿ',
 '[PAD]',
 'ㅎ',
 'ㄺ',
 'ㅖ',
 '[SEP]',
 'ㅣ',
 'ㄳ',
 'ㅌ',
 'ㄵ',
 '[UNK]',
 'ㄲ',
 '[CLS]',
 'ㄱ',
 'ㄸ',
 'ㅈ',
 '[MASK]',
 'ㅑ',
 'ㅙ',
 'ㅟ',
 'ㄶ',
 'ㅔ',
 'ㅅ',
 'ㅊ',
 'ㅘ',
 'ㅍ',
 'ㅞ',
 'ㄽ']

In [36]:
token

['ㄴ', 'ㅏ', ' ']

In [ ]:
class KoreanBPETokenizer:
    def __init__(self, num_merges=10):
        self.num_merges = num_merges
        self.tokenizer = Kkma() # 토크나이저를 포함해서 불러오는 부분
        self.word_freq = None
        self.byte_pairs = None
        self.token_vocab = None

    def train(self, corpus):
        self.word_freq = defaultdict(int)
        for sentence in corpus: # 문장을 형태소 변경하여 단어로 만들고 빈도 집계
            tokens = self.tokenizer.morphs(sentence)
            for token in tokens:
                self.word_freq[token] += 1

        for i in range(self.num_merges):
            self.byte_pairs = self.get_byte_pairs(self.word_freq)
            self.word_freq = self.merge_byte_pairs(self.byte_pairs, self.word_freq)

        self.token_vocab = set(self.word_freq.keys())

    def tokenize(self, text):
        tokens = self.tokenizer.morphs(text)
        tokenized_text = []
        for token in tokens:
            if token in self.token_vocab:
                tokenized_text.append(token)
            else:
                tokenized_text.extend(self.byte_tokenize(token))
        return tokenized_text

    def byte_tokenize(self, token):
        tokens = []
        while len(token) > 0:
            found = False
            for i in range(len(token), 0, -1):
                subword = token[:i]
                if subword in self.token_vocab:
                    tokens.append(subword)
                    token = token[i:]
                    found = True
                    break
            if not found:
                tokens.append(token[0])
                token = token[1:]
        return tokens

In [ ]:

# 토크나이저 학습
corpus = korean_train[:10]
tokenizer = KoreanBPETokenizer(num_merges=10)
tokenizer.train(corpus)



In [ ]:
# 문장 토크나이징
sentence ='같은 배우여도 독립영화냐 상업영화냐에 따라 생김새가 조금씩 다르게 보여'
tokens = tokenizer.tokenize(sentence)
print(tokens)